# 1- Drive Bağlantısı

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2- Kütüphaneler

In [ ]:
!pip install pymap3d pyproj

In [ ]:
# 1. Matematik ve Sayısal İşlemler
import math
import numpy as np
import scipy.optimize # Optimizasyon işlemleri için
from scipy.spatial import distance # İki nokta arasındaki mesafeleri hesaplamak için.
from scipy import signal #  Sinyal işleme fonksiyonları için
from scipy.interpolate import InterpolatedUnivariateSpline # Veriler üzerinde spline interpolasyonu yapmak için. Yani; noktalar arasında düzgün, kesintisiz ve pürüzsüz eğriler oluşturur.

# 2. Görselleştirme ve Veri Analizi
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px # Etkileşimli ve daha gelişmiş görselleştirmeler için.
import plotly.graph_objects as go # Etkileşimli ve daha gelişmiş görselleştirmeler için.

# 3. Coğrafi Hesaplamalar
import pymap3d as pm # GNSS konum verilerinin dönüştürülmesi ve Vincenty formülleri ile mesafe hesaplamaları için.
import pymap3d.vincenty as pmv #GNSS konum verilerinin dönüştürülmesi ve Vincenty formülleri ile mesafe hesaplamaları için.
import pyproj as proj # Coğrafi projeksiyonlar ve dönüşümler için.

# 4. Makine Öğrenimi ve Derin Öğrenme
from tensorflow import keras # Derin öğrenme modelleri oluşturmak ve eğitmek için.
from keras import layers # Yapay sinir ağları katmanlarını tanımlamak ve oluşturmak için .
from keras import models # Yapay sinir ağları modellerini tanımlamak ve oluşturmak için.

# 5. Dosya ve İlerleme Takibi
import glob as gl # Dosya sistemi ile çalışmak ve belirli desenlere uyan dosyaları bulmak için.
from tqdm.auto import tqdm # Uzun süren işlemlerde  ilerleme çubuğu oluşturmak için.

# 6. Gereksiz Uyarıları Görmeme
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Sabitler
CLIGHT = 299_792_458   # Işık Hızı (m/s). GNSS sinyallerinin uydu ile alıcı arasında hareket etme hızını temsil edecek.
RE_WGS84 = 6_378_137   # Dünya Yarıçapı (WGS84 yöntemiyle ölçülmüştür) (m). Dünya'nın WGS84 referans elipsoidine göre ekvator yarıçapını temsil edecek.
OMGE = 7.2921151467E-5  # Dünya'nın Açısal Hızı (rad/s). Dünya'nın kendi ekseni etrafındaki dönüş hızını temsil eder.  Konum hesaplamalarında kullanılır.

# 3- Uydu Seçimi

Bu fonksiyon, bir GNSS veri çerçevesinden (ör. device_gnss.csv dosyasından alınan veriler) belirli kriterlere göre uydu sinyallerini seçmek için kullanılacaktır.

Fonksiyon, aşağıdaki ölçütlere uymayan uydu sinyallerini filtreleyerek daha güvenilir veriler elde edilmesini sağlar:
- **Taşıyıcı frekans hatası (CarrierErrorHz) 2 MHz'ten küçük olmalı.** Çünkü, GNSS sinyallerinde taşıyıcı frekans hatasının küçük olması, sinyalin doğru bir şekilde alınması için önemlidir. 2 MHz üzerindeki hatalar genellikle sinyaldeki parazit veya sapmalardan kaynaklanır ve güvenilir veri elde edilmesini zorlaştırır.

- **Uydu eğim açısı (SvElevationDegrees) 10 dereceden büyük olmalı.** Çünkü, 10 derece altındaki açılar veya 15 derece üstündeki açılar, sinyalin atmosferik etkilerden (örneğin troposferik kırılma ya da bir diğer deyişle GPS uydusunun sinyalinin varışındaki gecikme) daha fazla etkilenmesine neden olabilir. Ayrıca, alıcıya gelen sinyalin zayıf olması düşük eğim açılarında daha olasıdır.

- **Taşıyıcı-gürültü oranı (Cn0DbHz) 15 dB-Hz'den büyük olmalı.** Çünkü, Taşıyıcı-gürültü oranı, sinyal kalitesini gösteren önemli bir metriktir. 15 dB-Hz altındaki sinyaller zayıf olarak kabul edilir ve konum belirleme doğruluğunu olumsuz etkileyebilir.

- **Çoklu yol etkisi (MultipathIndicator) olmadığını belirtmeli.** Çünkü, Çoklu yol etkisi, GNSS sinyallerinin yüzeylerden yansıması sonucu farklı yollardan alıcıya ulaşmasıdır. Bu durum, sinyalin doğruluğunu ciddi şekilde bozabilir. Bu nedenle, yalnızca doğrudan gelen sinyaller seçilmelidir.

Bu fonksiyonu yazmamızdaki sebep, GNSS sinyallerinin doğruluğunu artırmak ve hatalı sinyallerden kaynaklanan sapmaları azaltmaktır.

In [ ]:
# GNSS veri çerçevesinden uydu sinyallerini belirli kriterlere göre filtreleyen fonksiyon
def satellite_selection(df, column):
    """
    Bu fonksiyon, GNSS veri çerçevesinden belirli kriterlere uymayan uydu sinyallerini filtreler.

    Args:
        df : DataFrame from device_gnss.csv
            GNSS ölçümlerini içeren veri çerçevesi.
        column : str
            Sinyallerin filtreleneceği sütunun adı.

    Returns:
        df : DataFrame
            Belirtilen kriterlere uyan uydu sinyallerini içeren filtrelenmiş veri çerçevesi geri döndürülecek.
    """
    # Null olmayan değerlerle çalışmak için
    idx = df[column].notnull()

    # Taşıyıcı frekans hatası (CarrierErrorHz) 2 MHz'ten küçük olmalı. Çünkü:
    #  - GNSS sinyallerinde taşıyıcı frekans hatasının küçük olması, sinyali doğru bir şekilde almamız için önemlidir.
    #  - 2 MHz üzerindeki hatalar genellikle sinyaldeki parazit veya sapmalardan kaynaklanır ve güvenilir veriyi elde etmemizi zorlaştırır.
    idx &= df['CarrierErrorHz'] < 2.0e6

    # Uydu eğim açısı (SvElevationDegrees) 10 ila 15 derece arasında olmalı. Çünkü:
    #  - 10 derece altındaki açılar, sinyalin atmosferik etkilerden (örneğin troposferik kırılma - sinyaldeki gecikme) daha fazla etkilenmesine neden olabilir.
    #  - Ayrıca,  eğim açısı düştükçe alıcıya gelen sinyalin zayıflaması daha olasıdır.
    idx &= df['SvElevationDegrees'] > 10.0

    # Taşıyıcı-gürültü oranı (C/N0) 15 dB-Hz'den büyük olmalı. Çünkü:
    # - Taşıyıcı-gürültü oranı, sinyal kalitesini gösteren önemli bir metriktir.
    # - 15 dB-Hz altındaki sinyaller zayıf olarak kabul edilir ve konum belirleme doğruluğunu olumsuz etkileyebilir.
    idx &= df['Cn0DbHz'] > 15.0

    # Çoklu yol etkisi (MultipathIndicator = 0) olmamalı. Çünkü:
    # - Çoklu yol etkisi, GNSS sinyallerinin yüzeylerden yansıması sonucu farklı yollardan alıcıya ulaşmasıdır.
    # - Bu durum, sinyalin doğruluğunu ciddi şekilde bozabilir. Bu nedenle, yalnızca doğrudan gelen sinyaller seçilmeli , yüzeyden seken sinyalleri egale etmeliyiz.
    idx &= df['MultipathIndicator'] == 0

    # Belirtilen kriterlere uyan satırları döndür
    return df[idx]


# 4- Uydu Sinyallerinden Yeryüzündeki Bir Kullanıcının Konumunu ve Hızını Hesaplama (Weighted Least Squares / Ağırlıklı En Küçük Kareler Yönemi İle)

In [ ]:
def point_positioning(gnss_df):
    # Her uydu (Svid) ve sinyal tipi (SignalType) için taşıyıcı frekansı (CarrierFrequencyHz) sütunundaki medyan değerleri hesaplıyoruz.
    CarrierFrequencyHzRef = gnss_df.groupby(['Svid', 'SignalType'])['CarrierFrequencyHz'].median()
    # Yukarıda oluştruduğumuz her medyan değerini, orijinal GNSS verileriyle birleştirdik
    gnss_df = gnss_df.merge(CarrierFrequencyHzRef, how='left', on=['Svid', 'SignalType'], suffixes=('', 'Ref'))

    # Üstte Hesapladığım CarrierFrequencyHzRef ile veri setinde bulunan CarrierFrequencyHz arasındaki farkı bularak taşıyıcı frekansı hatasını (CarrierErrorHz) hesapladık ...
    # ... ve veri setimizde 'CarrierErrorHz' sütununu ekledik
    # Çünkü başlarda da bahsettiğimiz gibi Taşıyıcı frekans hatası (CarrierErrorHz) 2 MHz'ten küçük olursa işimize yarayacak
    gnss_df['CarrierErrorHz'] = np.abs((gnss_df['CarrierFrequencyHz'] - gnss_df['CarrierFrequencyHzRef']))

    # Pseudorange verisini (gerçek mesafe ölçümünden elde edilen değer) Carrier Smoothing (taşıyıcı-pürüzsüzleştirme) yöntemiyle düzgünleştirdik.
    # Basitçe Carrier Smoothing: pseudorange verisinin doğruluğunu artırmak için kullanılan bir tekniktir. Verilerdeki gürültüyü azaltır ve daha doğru yer belirleme (positioning) sağlar
    gnss_df = carrier_smoothing(gnss_df)

    # Benzersiz zaman damgalarını alacak ve toplam iterasyon sayısını hesaplayacağız. Yani, kaç farklı zaman noktasının (epoch) veri setinde bulunduğunu göstereceğiz
    utcTimeMillis = gnss_df['utcTimeMillis'].unique() # "utcTimeMillis", her bir veri kaydının zamanını milisaniye cinsinden temsil ediyor.
    nepoch = len(utcTimeMillis)  # Toplam iterasyon sayısı

    # Pozisyon ve hız için başlangıç vektörlerini tanımladık
    x0 = np.zeros(4)  # Başlangıç pozisyon [x, y, z, saat] -> her birine ilk değer olarak 0 atadık
    v0 = np.zeros(4)  # Başlangıç hız [vx, vy, vz, saat] -> her birine ilk değer olarak 0 atadık

    # Pozisyon ve hız tahminlerini kayıt etmek için matrisler oluşturacağız
    x_wls = np.full([nepoch, 3], np.nan)  # Pozisyon tahminlerini kaydetmek için
    v_wls = np.full([nepoch, 3], np.nan)  # Hız tahminlerini kaydetmek için


    for i, (t_utc, df) in enumerate(tqdm(gnss_df.groupby('utcTimeMillis'), total=nepoch)):
        # Geçerli uydu sinyallerini Pseudorange verisi için seçelim
        df_pr = satellite_selection(df, 'pr_smooth')
        # Geçerli uydu sinyallerini Pseudorange Rate verisi için seçelim
        df_prr = satellite_selection(df, 'PseudorangeRateMetersPerSecond')  # PseudorangeRateMetersPerSecond, pseudorange'nin zamanla değişme hızını (yani, alıcı ile uydu arasındaki mesafenin hızını) ifade eder.

        # GNSS verisindeki pseudorange hatalarını düzeltmek için gerekli düzenlemeler yapacağız:
        # - SvClockBiasMeters: Uydu saatinin yanlışı (saat kayması)
        # - IsrbMeters:  uydunun yerçekimi, ortam koşulları veya diğer etmenler nedeniyle oluşan küçük sapmaları ifade eder
        # - IonosphericDelayMeters (İyonosferik Gecikme): GNSS sinyalleri, iyonosferdeki serbest elektron yoğunluğu nedeniyle hız kaybına uğrar.
        # - TroposphericDelayMeters (Troposferik Gecikme): Troposferdeki nem, sıcaklık ve basınç faktörler,, GNSS sinyallerini etkileyebilir ve bu da sinyallerin alıcıya ulaşma süresini uzatır.
        pr = (df_pr['pr_smooth'] + df_pr['SvClockBiasMeters'] - df_pr['IsrbMeters'] - df_pr['IonosphericDelayMeters'] - df_pr['TroposphericDelayMeters']).to_numpy()

        # Pseudorange Rate (Pseudorange Hızı) ve SvClock Drift (Uydu Saat Kayması) verilerini birleştirilerek, düzeltilmiş Pseudorange Rate elde edeceğiz.
        prr = (df_prr['PseudorangeRateMetersPerSecond'] + df_prr['SvClockDriftMetersPerSecond']).to_numpy()

        # Uydu pozisyon verilerini Pseudorange verisi için alacağız:
        #  - ECEF (Earth-Centered, Earth-Fixed): Dünya merkezine dayalı bir koordinat sistemidir.
        xsat_pr = df_pr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters','SvPositionZEcefMeters']].to_numpy()

        # Uydu pozisyon verilerini Pseudorange Rate verisi için hız için alacağız
        xsat_prr = df_prr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters','SvPositionZEcefMeters']].to_numpy()

        # Uydu hız verilerini Pseudorange Rate verisi için alacağız
        vsat = df_prr[['SvVelocityXEcefMetersPerSecond', 'SvVelocityYEcefMetersPerSecond','SvVelocityZEcefMetersPerSecond']].to_numpy()

        # Pseudorange ve Pseudorange Rate için ağırlık matrislerini hesaplayacağız
        # Buradaki "1 / df_pr['RawPseudorangeUncertaintyMeters']" işlemi:
        # Her bir belirsizliğin tersini alır. Belirsizliğin tersten alınması, daha düşük belirsizliğe sahip ölçümlere daha fazla ağırlık verir...
        # ... Yüksek belirsizlik, düşük ağırlık anlamına gelir. Yani, güvenilir olmayan ölçümler daha düşük bir ağırlığa sahip olmalı.
        Wx = np.diag(1 / df_pr['RawPseudorangeUncertaintyMeters'].to_numpy())
        Wv = np.diag(1 / df_prr['PseudorangeRateUncertaintyMetersPerSecond'].to_numpy())


        # Artık kullanıcının POZİSYON TAHMİNİNİ yapabiliriz

        if len(df_pr) >= 4:  # Konumu doğru belirleyebilmek için en az 4 uydu gerekir -> 3D konum + zaman
            if np.all(x0 == 0):  # Başlangıç tahmini 0 ise
                # Bu kısımda, LEAST SQUARES (en küçük kareler) yöntemi kullanılarak pozisyon tahmini yapacağız:
                # - pr_residuals: Bu fonksiyon, modelle uydu ölçümleri arasındaki farkları hesaplayacak. (residuals = artık)
                # - jac_pr_residuals: ölçüm hatalarının (residul) türevini alarak doğrusal olmayan optimizasyonun yapılmasını sağlar.
                # - xsat_pr: Uyduların konum verilerini içeren NumPy dizisi.
                # - pr: Pseudorange (mesafe ölçüm) verileri.
                # - Wx: Pseudorange belirsizliklerine dayalı ağırlık matrisi.
                opt = scipy.optimize.least_squares(pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx))

                # Daha sonra yukarıda optimize edilen pozisyonu güncel pozisyonumuz olarak güncelledik
                x0 = opt.x

            # Normal WLS'ten sonra Robust (dayanıklı) WLS ile pozisyon tahmini yaparak ölçüm doğruluğunu arttıracağız:
            # Burada" soft_l1" loss fonksiyonu kullanılarak, pozisyon tahmini yapılırken daha az güvenilir ölçümlerin etkisi azaltılmak istiyoruz.
            opt = scipy.optimize.least_squares(pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx), loss='soft_l1')


            # Optimizasyonun durumunu kontrol ederek başarılı mı başarısız mı diye bakalım
            # - (0) = Başarılı optimizasyon
            # - (1) = Başarıyla tamamlanmış ancak istenen hassasiyete ulaşılmamış.
            # - (2) = Optimizasyon başarıyla tamamlanmış ancak maksimum iterasyon sayısına ulaşılmış. Yani, çözüme ulaşmak için daha fazla iterasyon yapılabilirdi ancak durdurulmuş.
            # - (-1): Başarısızlık, optimizasyonun çözüme ulaşamadığı durum.
            # - (-2): Başarısızlık.
            if opt.status < 1 or opt.status == 2:
                print(f'i = {i} KONUM optimizasyon durumu = {opt.status}')

            else:
              # Eğer optimizasyon başarılı bir şekilde tamamlanmışsa, tahmin edilen pozisyon (opt.x) alınır ve x_wls'ye kaydedilir.
              # Buradaki opt.x[:3] kısmı, elde edilen 4 elemanlı çözüm vektörünün (3D + Zaman) ilk 3 elemanını (x, y, z koordinatlarını) alır.
                x_wls[i, :] = opt.x[:3]  # Tahmin edilen pozisyonu kaydet
                x0 = opt.x  # Pozisyonu güncelle




        # Kullanıcının HIZ TAHMİNİNİ de yapabliriz

        # Üstte konum için yaptığımız tüm işlemlerin aynısını hız için de yapacağız
        if len(df_prr) >= 4:  # En az 4 uydu verisi gerekli demiştik
            if np.all(v0 == 0):  # Başlangıç tahmini yoksa normal WLS ile başla
                opt = scipy.optimize.least_squares(prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv))
                v0 = opt.x  # Optimize edilen hızı güncelledik
            # Robust (Dayanıklı) WLS ile hız tahmini yaptık
            opt = scipy.optimize.least_squares(prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv), loss='soft_l1')
            if opt.status < 1:  # Yİne optimizaston durumlarını kontrol edeceğiz
                print(f'i = {i} HIZ optimizasyon durumu = {opt.status}')
            else:
                v_wls[i, :] = opt.x[:3]  # Tahmin edilen hızı kaydet
                v0 = opt.x  # Hızı güncelle

    # Tahmin edilen zaman, pozisyon ve hız değerlerini döndür
    return utcTimeMillis, x_wls, v_wls


In [ ]:
# Carrier smoothing of pseudarange
def carrier_smoothing(gnss_df):

In [ ]:
# Compute line-of-sight vector from user to satellite
def los_vector(xusr, xsat):


In [ ]:
# Simple outlier detection and interpolation
def exclude_interpolate_outlier(x_wls, v_wls):


In [ ]:
# Kalman filter
def Kalman_filter(zs, us, phone):


In [ ]:
path = '/content/drive/MyDrive/BitirmeProjesi/smartphone-decimeter-2022/train/2021-07-19-US-MTV-1/GooglePixel4'
drive, phone = path.split('/')[-2:]

# Read data
gnss_df = pd.read_csv(f'{path}/device_gnss.csv')  # GNSS data
gt_df = pd.read_csv(f'{path}/ground_truth.csv')  # ground truth

# Point positioning
utc, x_wls, v_wls = point_positioning(gnss_df)

# Exclude velocity outliers
x_wls, v_wls = exclude_interpolate_outlier(x_wls, v_wls)

# Kalman smoothing
x_kf, _, _ = Kalman_smoothing(x_wls, v_wls, phone)

# Convert to latitude and longitude
llh_wls = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
llh_kf = np.array(pm.ecef2geodetic(x_kf[:, 0], x_kf[:, 1], x_kf[:, 2])).T

# Baseline
x_bl = gnss_df.groupby('TimeNanos')[
    ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']].mean().to_numpy()
llh_bl = np.array(pm.ecef2geodetic(x_bl[:, 0], x_bl[:, 1], x_bl[:, 2])).T

  0%|          | 0/1896 [00:00<?, ?it/s]